In [240]:
import cirq as cirq
from cirq.devices import GridQubit
from cirq import LineQubit, inverse
from cirq import Rx, Ry, Rz, Circuit, CNOT, TwoQubitMatrixGate
from cirq.google import ConvertToXmonGates

from pymps.fMPS import fMPS
from pymps.iMPS import iMPS
from pymps.spin import spins, U4, swap, swap_symmetrise, N_body_paulis
from pymps.spin import U4s, is_swap_symmetric, ferro, U1, U2, comm
from pymps.tensor import H as cT, C as c, haar_unitary, direct_sum, unitary_extension, T
from pymps.tensor import rotate_to_hermitian
from pymps.ncon import ncon

#import sympy as sp
from numpy.random import rand, randint, randn
from numpy import allclose, pad, real, ones,tensordot, array, block
from numpy import pi, arcsin, sqrt, transpose, prod, all, eye, zeros
from numpy import concatenate, kron, trace, sum, copy, diag
from functools import reduce
from numpy.linalg import qr, svd, cholesky, eigvals
from scipy.linalg import norm, null_space, expm, schur, polar
from scipy.optimize import approx_fprime as jac, minimize, NonlinearConstraint
Sx, Sy, Sz = spins(0.5)
Sx1, Sy1, Sz1 = N_body_paulis(0.5, 1, 2)
Sx2, Sy2, Sz2 = N_body_paulis(0.5, 2, 2)

import cirq
L = 5
D = 2
d = 2
A = iMPS().load('./A.npy')

def svals(A):
    q = svd(A)[1]
    return q/norm(q)

In [3]:
def random_unitary_gate(j=0, cse=False, depth=5, p=0.5):
    length = 2
    qubits = [GridQubit(j+i, 0) for i in range(length)]
    circuit = Circuit()
    if cse:
        '''random rotations and CNOTS 
           - converges to CUE as depth -> \infinity
           10.1103/PhysRevA.75.062314'''
        def U(i):
            """U: Random SU(2) element"""
            ψ = 2*π*rand()
            χ = 2*π*rand()
            φ = arcsin(sqrt(rand()))
            for g in [Rz(χ+ψ), Ry(2*φ), Rz(χ-ψ)]:
                yield g(GridQubit(i+j, 0))
        for i in range(depth):
            if rand()>p:
                # one qubit gate
                circuit.append(U(randint(0, length)))
            else:
                # two qubit gate
                if rand()>0.5:
                    circuit.append(CNOT(qubits[0], qubits[1]))
                else:
                    circuit.append(CNOT(qubits[1], qubits[0]))
        return circuit
    else:
        '''draw from SU(4) and implement gate directly'''
        U = qr(randn(4, 4)+1j*randn(4, 4))[0]
        circuit.append(TwoQubitMatrixGate(U)(qubits[0], qubits[1]))
        return circuit

def random_fmps(L, d=2, D=2, lr = 'r', cse=False, ti=False):
    '''create a random unitary circuit <=> finite mps'''
    if ti:
        qubits = [GridQubit(i, 0) for i in range(L)]
        circuit = Circuit()
        U = qr(randn(4, 4)+1j*randn(4, 4))[0]
        it = range(L-1) if lr=='r' else reversed(range(L-1))
        for i in it:
            circuit.append(TwoQubitMatrixGate(U)(qubits[i], qubits[i+1]))
        return circuit
    else:
        gates = [random_unitary_gate(j, cse=cse) for j in range(L)]
        
        if lr=='l':
            return reduce(lambda x, y: x+y, gates)
        else:
            return reduce(lambda x, y: x+y, reversed(gates))

def random_imps(d=2, D=2, lr='r', cse=False):
    '''create a random unitary circuit <=> infinite mps'''
    circuit = Circuit()
    return random_fmps(3, d, D, lr, cse, ti=True)

In [4]:
class fCirqMPS(fMPS):
    def __init__(self, data=None, d=None, D=None):
        '''initialise an fMPS'''
        super(fCirqMPS, self).__init__(data, d, D)
    
    def __repr__(self):
        return self.circuit.to_text_diagram(transpose=True, precision=3)
    
    def __str__(self):
        return self.circuit.to_text_diagram(transpose=True, precision=3)
    
    def copy(self):
        return fCirqMPS(self.data)
    
    def to_circuit(self, data=None):
        if data is None:
            data = self.data
        d, D = self.d, self.D
        self.circuit = Circuit()
        self.qbs = [GridQubit(i, 0) for i in range(L+1)]
        self.right_canonicalise()
        As = [transpose(a, [1, 0, 2]) for a in data]
        isometries = [a.reshape(-1, prod(a.shape[1:])) for a in As]
        unitaries = [unitary_extension(Q, 4) for Q in isometries]

        self.circuit.append([TwoQubitMatrixGate(u)(self.qbs[i], self.qbs[i+1]) for i, u in enumerate(unitaries)])
        return self
    
    def from_circuit(self, circuit=None):
        L, d, D = self.L, self.d, self.D
        z = array([1, 0])
        if circuit is None:
            circuit = self.circuit
        new_data = []
        for i, (moment, shape) in enumerate(zip(circuit, self.create_structure(L, d, D, True))):
            U = moment.operations[0].gate._matrix.reshape([d]*4)
            if i == 0:
                U_ = tensordot(tensordot(U, z, [0, 0]), z, [0, 0]).reshape(*shape)
                new_data.append(U_)
            elif i==L-1:
                U_ = tensordot(tensordot(U, z, [0, 0]), z, [1, 0]).transpose().reshape(*shape)
                new_data.append(U_)
            else:
                U_ = tensordot(U, z, [0, 0]).transpose([1, 0, 2]).reshape(*shape)
                new_data.append(U_)
        self.data = new_data
        return self
    
def test_fCirqMPS():  
    C = fCirqMPS().random(L, d, D).to_circuit();
    C_ = C.copy()
    assert C.from_circuit() == C_
    print('Complete')

test_fCirqMPS()

Complete


In [119]:
class iCirqMPS(iMPS):
    def __init__(self, data=None, canonical=None):
        '''initialise an iMPS'''
        super(iCirqMPS, self).__init__(data, canonical)
        self.period = 1
        self.d = 2
        self.D = 2
        
    def __repr__(self):
        if hasattr(self, 'circuit'):
            return self.circuit.to_text_diagram(transpose=True)
        else:
            return 'iCirqMPS'
    
    def from_circuit(self, circuit=None):
        '''takes a (right orthogonal) circuit and returns a (right orthogonal) iMPS representation'''
        d, D = self.d, self.D
        shape = self.create_structure(d, D)[0]

        z = array([1, 0])
        if circuit is None:
            circuit = self.circuit
        U = circuit[0].operations[0].gate._matrix.reshape([d]*4)
        self.Us = [U]
        U_ = tensordot(U, z, [0, 0]).transpose([1, 0, 2])
    
        self.data = [U_]
        return self
    
    def to_operations(self, data=None):
        '''takes a right (orthogonal) mps and doesn't check. need to implement canonical form for iMPS.
           canonicalisation is not idempotent!'''
        if data is None:
            data = self.data  
        d, D = self.d, self.D
        As = [transpose(a, [1, 0, 2]) for a in data]
        isometries = [a.reshape(-1, prod(a.shape[1:])) for a in As]
        self.Us = [unitary_extension(Q, 4) for Q in isometries]
        self.ops = [TwoQubitMatrixGate(u) for u in unitaries]
        return self
    
    def to_circuit(self, data=None):
        self.to_operations(data)
        self.circuit = Circuit()
        self.qbs = [LineQubit(0), LineQubit(1)]
        self.circuit.append([op(self.qbs[i], self.qbs[i+1]) for i, op in enumerate(self.ops)])
        return self
    
    def from_unitaries(self, Us):
        self.Us = Us
        self.ops = [TwoQubitMatrixGate(u) for u in Us]
        self.qbs = [LineQubit(0), LineQubit(1)]
        self.circuit = Circuit()
        self.circuit.append([op(self.qbs[i], self.qbs[i+1]) for i, op in enumerate(self.ops)])
        self.from_circuit()
        return self
    
    def add_env_op(self, v0=None):
        '''(D=2): get the environment on the circuit'''
        if v0 is None:
            v0 = randn(8)
        if not hasattr(self, 'ops'):
            if hasattr(self, 'data'):
                self.to_operations()
            else:
                raise Exception("need data")
                
        def obj(v):
            '''takes a vectorized real rep of 2x2 matrix
            return --|v|-----|u|------1
                   --|v|-|u|-|u|-|v|--1
                   ------|u|-----|v|--1'''
            v = embed(mat(v))
            u, v = TwoQubitMatrixGate(U), TwoQubitMatrixGate(v)

            qbs = [LineQubit(i) for i in range(3)]
            uv, vu, inv_vu, inv_uv = Circuit(), Circuit(), Circuit(), Circuit()

            uv.append([v(qbs[1], qbs[2]), u(qbs[0], qbs[1])])
            inv_uv.append([inverse(u(qbs[0], qbs[1])), inverse(v(qbs[1], qbs[2]))])

            vu.append([v(qbs[0], qbs[1]), u(qbs[1], qbs[2])])
            inv_vu.append([inverse(u(qbs[1], qbs[2])), inverse(v(qbs[0], qbs[1]))])

            C = (vu+inv_uv)
            i = C.apply_unitary_effect_to_state(0b000)

            upupup = kron(kron(array([1, 0]), array([1, 0])), array([1, 0]))
            return norm(upupup-i)
         
        res = minimize(obj, v0, method='Powell')
        return res
        
def test_iCirqMPS():
    v = array([1, 2, 3, 4, 5, 6, 7, 8, 9])
    Us = [U4s(v)]
    print(is_swap_symmetric(Us[0]))
    
    A = iCirqMPS().from_unitaries(Us)    
    A.add_env_op()
    print('Complete')

test_iCirqMPS()

True


ValueError: Not a 4x4 unitary matrix: [[[[ 0.81825693+0.00713262j  0.12083766+0.22578553j]
   [ 0.12083766+0.22578553j -0.42747031+0.12849969j]]

  [[-0.28787169+0.18821345j  0.7348349 -0.23592573j]
   [ 0.05926393+0.50136924j -0.16233265+0.0694773j ]]]


 [[[-0.28787169+0.18821345j  0.05926393+0.50136924j]
   [ 0.7348349 -0.23592573j -0.16233265+0.0694773j ]]

  [[ 0.1714275 +0.25382773j  0.0292104 +0.28816541j]
   [ 0.0292104 +0.28816541j  0.7687343 +0.38398771j]]]]

In [120]:
from numpy import tensordot as td, real, imag
# 2 3
# | |
# ---
# | |
# 0 1

# 2i I
# |  |
# ----
# |  |
# 0s 1j

# I 3j
# |  |
# ----
# |  |
# 0i 1s

In [284]:
def is_unitary(u):
    return allclose(eye(u.shape[0]), u@u.conj().T)

def embed(v):
    '''put a matrix into a unitary'''
    v = v.reshape(1, -1)/norm(v)
    vs = null_space(v).conj().T
    return concatenate([v, vs], 0).T

def deembed(u):
    '''take a matrix out of a unitary'''
    return (u@array([1, 0, 0, 0])).reshape(2, 2)

def mat(v):
    re, im = v[:4], v[4:]
    C = (re+im*1j).reshape(int(sqrt(len(v))), -1)
    return C

def demat(A):
    re, im = real(A).reshape(-1), imag(A).reshape(-1)  
    return concatenate([re, im], axis=0)

Q = randn(2, 2)+1j*randn(2, 2)

assert is_unitary(embed(Q))
assert allclose(Q, deembed(embed(Q))*norm(Q))
assert allclose(norm(mat(demat(Q))-Q), 0)

SWAP = ((Sx1@Sx2+Sy1@Sy2+Sz1@Sz2)+eye(4))/2

In [666]:
H = 4*(Sx1@Sx2+Sy1@Sy2+Sz1@Sz2).reshape(2, 2, 2, 2) 
x = randn(9)
U = U4s(x)
C, AL, AR = to_mps(U)

In [699]:
def to_mps(U):
    U = U.reshape(2, 2, 2, 2)
    z = array([1, 0])

    # take a unitary, return [AL, AR, C]
    AL = td(U, z, [3, 0]).transpose([0, 2, 1]) # -> (s, j, i) -> (s, i, j)
    AR = td(U, z, [2, 0]).transpose([1, 0, 2]) # -> (i, s, j) -> (s, i, j)


    assert allclose(norm(AL-T(AR)), 0)

    def obj(v):
        C = mat(v)
        return norm(AL@C-C@AR)

    res = minimize(obj, randn(8), method='Powell')
    P = mat(res.x)
    P /= norm(P)

    A = iMPS([copy(AR)])
    _, l, r = A.transfer_matrix().eigs()
    L, R = cholesky(l), cholesky(r)
    Q = L@R
    assert allclose(svals(Q)/norm(svals(Q)), svals(P/norm(svals(P))))
    #print(res.fun, svals(Q)/norm(svals(Q)), svals(P)/norm(svals(P)), sep='\n')
    # P comes from minimization procedure, Q from eigenvalues. 
    # P is correct, (in that it pulls through) but only unitarily the same as Q
    # Q doesn't pull through?

    assert res.fun<1e-10

    assert allclose(sum(AL@cT(AL), 0), eye(2))
    assert not allclose(sum(AR@cT(AR), 0), eye(2))

    assert allclose(sum(cT(AR)@AR, 0), eye(2))
    assert not allclose(sum(cT(AL)@AL, 0), eye(2))
    return [P, AL, AR]

def ev1(U, op):
    P, AL, AR = to_mps(U)
    AC = AR@P
    AC_ = td(op, AC, [1, 0])
    return real(sum(AC@cT(AC_), 0).trace())

def ev2(U, op):
    P, AL, AR = to_mps(U)
    AC = AR@P
    return real(ncon([c(AC), c(AR), op, AC, AR], [[1, 5, 6], 
                                                  [2, 6, 7], 
                                                  [1, 2, 3, 4], 
                                                  [3, 5, 8], 
                                                  [4, 8, 7]]))

def minIsing(U, rep=100):
    qbs = [LineQubit(i) for i in range(4)]
    for _ in range(1):
        P, _, _ = to_mps(U)
        V = embed(P)
        U, V = TwoQubitMatrixGate(U), TwoQubitMatrixGate(V)
        c = Circuit()
        c.append([V(qbs[1], qbs[2]), 
                  U(qbs[0], qbs[1])])
        c.append(cirq.measure(qbs[1], key='x'))
        
        simulator = cirq.google.XmonSimulator()
        res = simulator.run(c, repetitions=rep)
        data = res.measurements['x'].astype(int)  
        
    zI = data.mean(0)
    return zI


In [700]:
from numpy import pi
from scipy.linalg import expm
U = U4s(x)
minIsing(U, 1000), ev1(U, Sz)

(array([0.44]), 0.06929725750784357)

In [13]:
U = U.reshape(4, 4)
def qobj(v):
    '''takes a vectorized real rep of 2d matrix'''
    v = embed(mat(v))
    u, v = TwoQubitMatrixGate(U), TwoQubitMatrixGate(v)

    qbs = [LineQubit(i) for i in range(3)]
    uv, vu, inv_vu, inv_uv = Circuit(), Circuit(), Circuit(), Circuit()

    uv.append([v(qbs[1], qbs[2]), u(qbs[0], qbs[1])])
    inv_uv.append([inverse(u(qbs[0], qbs[1])), inverse(v(qbs[1], qbs[2]))])
    
    vu.append([v(qbs[0], qbs[1]), u(qbs[1], qbs[2])])
    inv_vu.append([inverse(u(qbs[1], qbs[2])), inverse(v(qbs[0], qbs[1]))])

    C = (vu+inv_uv)
    i = C.apply_unitary_effect_to_state(0b000)
    
    upupup = kron(kron(array([1, 0]), array([1, 0])), array([1, 0]))
    return norm(upupup-i)

In [14]:
F = minimize(qobj, randn(8), method='BFGS', tol=0.01)
svals(mat(F.x))/norm(svals(mat(F.x)))

array([0.91416536, 0.40534146])

In [352]:
def f(x):
    def qobj_measure(v):
        '''takes a vectorized real rep of 2d matrix'''
        v = embed(mat(v))
        u, v = TwoQubitMatrixGate(U), TwoQubitMatrixGate(v)

        qbs = [LineQubit(i) for i in range(3)]
        uv, vu, inv_vu, inv_uv = Circuit(), Circuit(), Circuit(), Circuit()

        uv.append([v(qbs[1], qbs[2]), u(qbs[0], qbs[1])])
        inv_uv.append([inverse(u(qbs[0], qbs[1])), inverse(v(qbs[1], qbs[2]))])

        vu.append([v(qbs[0], qbs[1]), u(qbs[1], qbs[2])])
        inv_vu.append([inverse(u(qbs[1], qbs[2])), inverse(v(qbs[0], qbs[1]))])

        C = (vu+inv_uv)
        C.append([cirq.measure(*qbs, key='x')])
        return C

    simulator = cirq.google.XmonSimulator()
    res = simulator.run(qobj_measure(x), repetitions=1000)
    data = res.measurements['x'].astype(int)
    return norm(data.mean(axis=0)-array([1, 0, 0]))

In [355]:
from tqdm import tqdm_notebook as tqdm
def grad(f, x, eps):
    e = eye(len(x))
    J = zeros(len(x))
    for i in range(len(x)):
        J[i] = (f(x+eps*e[i])-f(x))/eps
    return J

def gd(f, x0, eps, n_iters=100):
    xs = [x0]
    for i in tqdm(range(n_iters)):
        xs.append(xs[-1]-eps*grad(f, xs[-1], eps))
    print(f(xs[-1]))
    return xs

In [356]:
xs = gd(f, randn(8), 0.1)

KeyboardInterrupt: 

In [ ]:
ys = gd(qobj, randn(8), 0.1)

In [ ]:
svals(mat(xs[-1]))

In [ ]:
svals(mat(ys[-1]))

In [405]:
C = [LineQubit(0), LineQubit(1)]
u = TwoQubitMatrixGate(qr(randn(4, 4))[0])(*C)
c = Circuit()
c.append(u)

In [406]:
cirq.google.ConvertToXmonGates()(c)

In [407]:
c

0: ───Y^0.194───@───Y^0.5────@─────────PhasedX(-0.629)^0.5───@──────────PhasedX(0.632)^0.5──────Z^0.368────
                │            │                               │
1: ───Y^0.573───@───X^-0.5───@^0.742───PhasedX(-0.129)^0.5───@^-0.677───PhasedX(-0.967)^0.935───Z^(7/15)───